In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# TITANIC SURVIVAL PREDICTION USING DIFFERENT ML MODELS
**Through this project I aim to compare the performance metrics of different ML algorithms on the titanic dataset. The different ML models used for comparison are as follows**
1. **Logistic Regression**
2. **Decision Trees**
3. **Random Forests**

The plan of action is as follows:
1. **DATA CLEANING**
2. **EXPLORATORY DATA ANALYSIS**
3. **FEATURE ENGINEERING**
4. **MODEL TRAINING AND EVALUATION**
5. **COMPARISON OF EVALUATION METRICS AMONGST DIFFERENT MODELS**



In [ ]:
## LOADING THE DATASET AND GETTING AN OVERVIEW OF THE DATA
df=pd.read_csv('/kaggle/input/titanic/Titanic-Dataset.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
## CHECKING FOR NULL VALUES
df.isnull().sum()

In [ ]:
## REMOVING THE CABIN COLUMN FROM THE DATASET AS IT IS INSIGNIFICANT AND CONTAINS MANY NULL VALUES
df.drop('Cabin',axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
age_median=df.Age.median()
print(age_median)

In [ ]:
df['Age']=df['Age'].fillna(age_median)

In [ ]:
df['Age'].isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df['Embarked']=df['Embarked'].fillna('S')

In [ ]:
## CHECKING FOR DUPLICATED ROWS
df.duplicated().sum()

In [ ]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1


In [ ]:
## EDA TO FIND OUT FACTORS ACTUALLY AFFECTING THE SURVIVAL 
df.columns

In [ ]:
## AGE DISTRIBUTION OF THOSE WHO SURVIVED AND THOSE WHO DIDN'T
sns.distplot(df[df['Survived']==1]['Age'],hist=False)
sns.distplot(df[df['Survived']==0]['Age'],hist=False)


In [ ]:
## SURVIVAL BY GENDER
sns.barplot(x="Sex", y="Survived", data=df)
plt.title('Gender Distribution According to Survival ')
plt.show()

More no of females survived as compared to men

In [ ]:
## SURVIVAL BY PCLASS
sns.barplot(x="Pclass", y="Survived", data=df)
plt.title('Survival Ratio by PClass')
plt.show()

The survival rate of passengers travelling in Class 1 is the highest and that of Class 3 is the lowest.

In [ ]:
## SURVIVAL BY EMBARKED
sns.barplot(x="Embarked", y="Survived", data=df)
plt.show()

In [ ]:
## SURVVIAL BY FARE
df['Fare_Range'] = pd.qcut(df['Fare'], 4)
ax=sns.barplot(data=df,x='Fare_Range',y='Survived')
plt.title('Survival Ratio based on Fare')
plt.show()

The passengers who paid higher fare had higher chance of survival.

In [ ]:
sns.barplot(x="FamilySize", y="Survived", data=df)

**Passengers with very large families had lower survival chances (harder to escape).**

**Passengers with small families had better chances than those traveling alone.**

In [ ]:
## CORRELATION OF NUMERIC FEATURES
columns = []
for col in df.columns:
    if df[col].dtype == "int64" or df[col].dtype == "float64":
        columns.append(col)
columns

In [ ]:
columns.remove('PassengerId')

In [ ]:
columns

In [ ]:
df[columns].corr()["Survived"].sort_values(ascending=False)


In [ ]:
pd.crosstab(df["Sex"], df["Pclass"], values=df["Survived"], aggfunc="mean")


Females in 1st class had highest survival, males in 3rd had lowest.”

**The survival of passenger primarily depends upon the following features:**
1. Fare
2. Age
3. Gender
4. PClass
5. Embarked
6. FamilySize


In [ ]:
df.columns

In [ ]:
df.Sex=df.Sex.map({'male':1,'female':2})

In [ ]:
df = pd.get_dummies(df, columns=["Embarked"], drop_first=True)

In [ ]:
## DATA IS READY FOR ML MODELS

X=df.drop(['PassengerId','Survived','Name','SibSp','Parch','Ticket','Fare_Range'],axis=1)
X.columns              ## INPUT FEATURES

In [ ]:
y=df['Survived']   ## TARGET FEATURE

In [ ]:
## COMPARING DIFFERENT MODELS BY THERE PERFORMANCE METRICS
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(max_depth=4,ccp_alpha=0.01),
    "Random Forest": RandomForestClassifier()
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_prob)
    })


results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
## RECALL COMPARISON OF DIFFERENT MODELS
sns.barplot(x="Model", y="Recall", data=results_df)
plt.show()

In [ ]:
## PRECISION COMPARISON OF DIFFERENT MODELS
sns.barplot(x="Model", y="Precision", data=results_df)
plt.show()

In our case the metrics **Precision** and **Recall** cater to our needs well and should be given more importance . But first we must understand how we can interpret these terms
* **Precision-** Tells us out of predicted survivors, how many really survived.
* **Recall-**  Tells us out of all real survivors, how many we correctly predicted.
- These parameters give us a better measure of the model's efficiency because they show a picture of survivors.

1. Out of all the models **Random Tree** performs the best and gives the highest recall value.
2. Out of all the models **Decision Tree** performs the best and gives the highest precision value.

In [ ]:
from sklearn.metrics import RocCurveDisplay
for name, model in models.items():
    RocCurveDisplay.from_estimator(model, X_test, y_test)
    plt.title(f'{name} ROC Curve')
plt.show()

**ROC = Receiver Operating Characteristic curve**

It shows how well our model can separate the two classes (e.g. Survived vs Not Survived).
* **X-axis = False Positive Rate (FPR)**
- Out of non-survivors, how many did we wrongly predict as survivors?
* **Y-axis = True Positive Rate (TPR) = Recall**
-Out of real survivors, how many did we correctly predict as survivors?

**AUC=Area Under Curve**
- AUC is the probability that the model ranks a random survivor higher than a random non-survivor.
- It’s a single number that summarizes how good the ROC curve is.
- It ranges between 0 and 1 , larger the value of AUC better is the classification result

## CONCLUSION
* In our dataset **Random Forest** algorithm gives the highest value of AUC hence it is the best model for prediction.